In [45]:
import featuretools as ft
import duckdb
con =  duckdb.connect("/home/manat/projects/obsq/work/dev/data/data.duckdb") 

In [46]:
observations_df = con.execute("SELECT * FROM preprocessed.gbif_citizen").df()
users_df = con.execute("SELECT * FROM preprocessed.inat_observers").df()
env_df = con.execute("SELECT * FROM raw.gee_citizen_occurences").df()
con.close()

In [47]:
users_df.head(1)

,id,login,spam,suspended,created_at,login_autocomplete,login_exact,name,name_autocomplete,orcid,icon,observations_count,identifications_count,journal_posts_count,activity_count,species_count,universal_search_rank,roles,site_id,icon_url
0,5355526,"""a_dumas""",false,false,"""2022-02-10T22:33:49+00:00""","""a_dumas""","""a_dumas""","""Alexandre Dumas""","""Alexandre Dumas""",null,"""https://static.inaturalist.org/attachments/us...",3312,2796,0,6108,1101,3312,[],1,"""https://static.inaturalist.org/attachments/us..."


In [48]:
users_df = users_df.drop(columns = ['name_autocomplete', 'login', 'login_autocomplete'], axis = 1)

In [49]:
dataframes = { 'observations' : (observations_df, 'gbifID'),
                'users' : (users_df, 'name'),
                'env' : (env_df, 'gbifID')
}


In [50]:
for df, index in dataframes.values():
    if not df[index].is_unique:
        #duplicateRows = df[index][df[index].duplicated()]
        rows = df[df[index].duplicated(keep=False)]
  
        for key, group in df[df[index].duplicated(keep=False)].groupby(index):
            print("Conflict on:", key)
            print(group)
            print("-" * 20)


Conflict on: ""
           id   spam suspended                   created_at      login_exact  \
546     38530  false     false  "2014-05-14T05:24:25+00:00"    "kevinhintsa"   
573   8109576  false     false  "2024-05-06T05:45:56+00:00"   "lauriane2638"   
897   1517204  false     false  "2019-03-09T06:15:19+00:00"      "shelley_b"   
1045  4108630  false     false  "2021-03-16T00:37:32+00:00"          "a9595"   
1058  8347928  false     false  "2024-07-10T11:05:31+00:00"      "abymystic"   
...       ...    ...       ...                          ...              ...   
4215  8340390  false     false  "2024-07-07T21:28:22+00:00"   "zachary-diab"   
4219  3627467  false     false  "2020-09-17T20:23:30+00:00"  "zakariabahous"   
4227  3612203  false     false  "2020-09-15T10:09:00+00:00"       "zemleroi"   
4237  6090040  false     false  "2022-09-01T02:31:01+00:00"      "zuhalhssn"   
4238   109812  false     false  "2015-06-13T17:04:37+00:00"         "zumbio"   

     name orcid        

In [14]:
relationships = [ ("observations", 'recordedBy', 'users', 'name')]

In [15]:
feature_matrix_users, features_defs = ft.dfs(
dataframes=dataframes,
relationships=relationships,
target_dataframe_name="observations",
)
feature_matrix_users


/home/manat/miniconda3/envs/obsq/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/manat/miniconda3/envs/obsq/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/manat/miniconda3/envs/obsq/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/home/manat/miniconda3/envs/obsq/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not

IndexError: Index column must be unique